In [104]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../DATA/gene_expression.csv')

In [3]:
df.head()

Gene One  Gene Two  Cancer Present
0       4.3       3.9               1
1       2.5       6.3               0
2       5.7       3.9               1
3       6.1       6.2               0
4       7.4       3.4               1

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gene One        3000 non-null   float64
 1   Gene Two        3000 non-null   float64
 2   Cancer Present  3000 non-null   int64  
dtypes: float64(2), int64(1)
memory usage: 70.4 KB


In [5]:
df.describe()

Gene One     Gene Two  Cancer Present
count  3000.000000  3000.000000     3000.000000
mean      5.600133     5.410467        0.500000
std       1.828388     1.729081        0.500083
min       1.000000     1.000000        0.000000
25%       4.300000     4.000000        0.000000
50%       5.600000     5.400000        0.500000
75%       6.900000     6.700000        1.000000
max      10.000000    10.000000        1.000000

In [10]:
df['Cancer Present'].value_counts()

1    1500
0    1500
Name: Cancer Present, dtype: int64

In [40]:
len(df)

3000

In [114]:
fig, ax = plt.subplots(dpi=150)
sns.scatterplot(data=df, x='Gene One',y='Gene Two',hue='Cancer Present',alpha=0.5)
plt.xlim(2,6)
plt.ylim(4,8);

In [123]:

sns.pairplot(data = df,hue='Cancer Present',height=5);

In [124]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [126]:
X = df.drop('Cancer Present',axis = 1)
X

Gene One  Gene Two
0          4.3       3.9
1          2.5       6.3
2          5.7       3.9
3          6.1       6.2
4          7.4       3.4
...        ...       ...
2995       5.0       6.5
2996       3.4       6.6
2997       2.7       6.5
2998       3.3       5.6
2999       4.6       8.2

[3000 rows x 2 columns]

In [127]:
y = df['Cancer Present']

In [153]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [154]:
scaler = StandardScaler()

In [155]:
scaled_X_train = scaler.fit_transform(X_train)

In [156]:
scaled_X_test = scaler.transform(X_test)

In [157]:
from sklearn.neighbors import KNeighborsClassifier

In [158]:
knn_model = KNeighborsClassifier(n_neighbors=1)

In [159]:
knn_model.fit(scaled_X_train, y_train)

KNeighborsClassifier(n_neighbors=1)

In [160]:
y_pred = knn_model.predict(scaled_X_test)

In [161]:
from sklearn.metrics import confusion_matrix, classification_report

In [162]:
confusion_matrix(y_test, y_pred)

array([[420,  50],
       [ 47, 383]])

In [163]:
len(y_test)

900

In [164]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.90      0.89      0.90       470
           1       0.88      0.89      0.89       430

    accuracy                           0.89       900
   macro avg       0.89      0.89      0.89       900
weighted avg       0.89      0.89      0.89       900



In [165]:
df['Cancer Present'].value_counts()

1    1500
0    1500
Name: Cancer Present, dtype: int64

In [182]:
test_error_rates = []

for k in range(1,30):
    knn_model = KNeighborsClassifier(n_neighbors = k)
    knn_model.fit(scaled_X_train, y_train)
    
    #create predictions
    y_pred = knn_model.predict(scaled_X_test)
    error = 1 - (accuracy_score(y_test, y_pred))
    test_error_rates.append(error)
    

In [181]:
from sklearn.metrics import accuracy_score

In [185]:
test_error_rates

[0.10777777777777775,
 0.09999999999999998,
 0.07444444444444442,
 0.07666666666666666,
 0.07222222222222219,
 0.06444444444444442,
 0.06444444444444442,
 0.06222222222222218,
 0.06000000000000005,
 0.06222222222222218,
 0.06222222222222218,
 0.06000000000000005,
 0.06222222222222218,
 0.06222222222222218,
 0.06000000000000005,
 0.05666666666666664,
 0.05555555555555558,
 0.05555555555555558,
 0.053333333333333344,
 0.053333333333333344,
 0.054444444444444406,
 0.05111111111111111,
 0.054444444444444406,
 0.054444444444444406,
 0.05666666666666664,
 0.05666666666666664,
 0.05555555555555558,
 0.05777777777777782,
 0.05777777777777782]

In [188]:
k_errors = pd.Series(data=test_error_rates, index = np.arange(1,30))

In [193]:
plt.plot(k_errors.index, k_errors)
plt.ylabel('ERROR RATE')
plt.xlabel('K Neighbors')

Text(0.5, 0, 'K Neighbors')

In [194]:
# PIPELINE --> GRIDSEARCH CV

In [249]:
scaler = StandardScaler()

In [250]:
knn = KNeighborsClassifier()

In [251]:
knn.get_params().keys()

dict_keys(['algorithm', 'leaf_size', 'metric', 'metric_params', 'n_jobs', 'n_neighbors', 'p', 'weights'])

In [252]:
operations = [('scaler',scaler),('knn',knn)]

In [253]:
from sklearn.pipeline import Pipeline

In [254]:
pipeline = Pipeline(operations)

In [255]:
from sklearn.model_selection import GridSearchCV

In [256]:
k_values = list(range(1,20))

In [257]:
k_values

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [258]:
param_grid = {'knn__n_neighbors':k_values}

In [259]:
from sklearn.metrics import SCORERS
sorted(list(SCORERS.keys()))

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_we

In [260]:

full_cv_classifier = GridSearchCV(pipeline,  param_grid, cv = 5, scoring = 'accuracy')

In [261]:
full_cv_classifier.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                              12, 13, 14, 15, 16, 17, 18, 19]},
             scoring='accuracy')

In [263]:
full_cv_classifier.best_estimator_.get_params()

{'memory': None,
 'steps': [('scaler', StandardScaler()),
  ('knn', KNeighborsClassifier(n_neighbors=14))],
 'verbose': False,
 'scaler': StandardScaler(),
 'knn': KNeighborsClassifier(n_neighbors=14),
 'scaler__copy': True,
 'scaler__with_mean': True,
 'scaler__with_std': True,
 'knn__algorithm': 'auto',
 'knn__leaf_size': 30,
 'knn__metric': 'minkowski',
 'knn__metric_params': None,
 'knn__n_jobs': None,
 'knn__n_neighbors': 14,
 'knn__p': 2,
 'knn__weights': 'uniform'}

In [265]:
full_pred = full_cv_classifier.predict(X_test)

In [266]:
print(classification_report(y_test, full_pred))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94       470
           1       0.95      0.92      0.93       430

    accuracy                           0.94       900
   macro avg       0.94      0.94      0.94       900
weighted avg       0.94      0.94      0.94       900



In [267]:
new_patient = [[3.8,6.4]]

In [268]:
full_cv_classifier.predict(new_patient)

array([0])

In [269]:
full_cv_classifier.predict_proba(new_patient)

array([[1., 0.]])